In [ ]:
'''
This cluster of functions allows a user to iterate through a designated field within a feature 
class, select all features for each unique attribute of that designated field, and export those
features as their own feature class to a designated geodatabase. 
'''

In [7]:
'''
Parameters
'''
# The feature class to be iterated through 
input_path      = ""

# The designated field 
attribute_field = ""

# The output geodatabase 
gdb             = ""

In [ ]:
'''
This creates a dictionary of feature class names (attribute) and field values.
The field values will be used to populate SQL clauses.
'''

def create_selection_dictionary(input_path, attribute_field): 
    
    # Create selection dictionary
    selection_dictionary = {} 

    # Iterate through feature class to populate dictionary
    with arcpy.da.SearchCursor(input_path, [attribute_field]) as cursor: 
        for row in cursor: 

            # Identify attribute 
            attribute = row[0]
            attribute_name = attribute.replace(" ", "_")
            print(attribute, attribute_name)

            # Populate dictionary 
            selection_dictionary[attribute_name] = attribute
        
    return selection_dictionary

selection_dictionary = create_selection_dictionary(input_path, attribute_field)

'''
Export through the dictionary 
'''

def export_from_selection_dictionary(selection_dictionary, gdb): 
    
    # Iterate through selection dictionary 
    for attribute_name, attribute in selection_dictionary.items(): 

        # Assemble SQL clause 
        sql = f"{attribute_field} = '{attribute}'"

        # Define output path 
        output_path = os.path.join(gdb, attribute_name)

        # Export 
        arcpy.conversion.ExportFeatures(input_path, output_path, sql) 
    
    return gdb 

'''
Combine both functions 
'''

def export_features_by_attribute(input_path, attribute_field, gdb): 
    
    selection_dictionary = create_selection_dictionary(input_path, attribute_field)
    gdb                  = export_from_selection_dictionary(selection_dictionary, gdb)
    
    return gdb 